In [ ]:
!ls
%cd ../..
!ls
# Now you should see the characters-and-dialouges-association-in-comics directory

In [3]:
import os
import subprocess
from pathlib import Path

END_WITH_LOCAL = 'characters-and-dialouges-association-in-comics'

os.environ['PATH'] = f"/root/.cargo/bin:{os.environ['PATH']}"

BASE_DIR = str(Path(os.getcwd()).parent.parent)
print(f"BASE_DIR: {BASE_DIR}")

# Simple validation
if not (BASE_DIR.endswith('/content') or BASE_DIR.endswith(END_WITH_LOCAL)):
    raise ValueError(f"Expected to be in .../{END_WITH_LOCAL} or .../content directory, but got: {BASE_DIR}")

BASE_DIR: e:\B3\group_prj\characters-and-dialouges-association-in-comics


In [4]:
import os
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"

import torch
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")
print(f"Using device: {device}")


Using device: cuda


In [9]:
import os
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
from detectron2.data.datasets import register_coco_instances
from detectron2.modeling import build_model
from detectron2.checkpoint import DetectionCheckpointer

# --- 1. Re-register the datasets (important if in a new script/session) ---
base_path = os.path.join(BASE_DIR, "data", "Manga109", "Human_Annotate_300")
images_dir = os.path.join(base_path, "train")
train_json_path = os.path.join(base_path, "train.json")
val_json_path = os.path.join(base_path, "val.json")

# Register the datasets
register_coco_instances("manga109_train", {}, train_json_path, images_dir)
register_coco_instances("manga109_val", {}, val_json_path, images_dir)


# --- 2. Setup Configuration ---
cfg = get_cfg()
# Use the same config file as for training

cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))

# Set the device
cfg.MODEL.DEVICE = str(device)

# Point to the model weights you just trained
model_output_dir = os.path.join(BASE_DIR, "models", "bubble-detection", "detectron2")
cfg.MODEL.WEIGHTS = os.path.join(model_output_dir, "model_final.pth")

# Set the number of classes (must match your training config)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # bubble

# Set the test dataset
cfg.DATASETS.TEST = ("manga109_val",)

# Set a confidence threshold for this evaluation
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7  # You can adjust this threshold


# --- 3. Build the Model and Run Evaluation ---

# Build the model from the config
model = build_model(cfg)
# Load the weights
DetectionCheckpointer(model).load(cfg.MODEL.WEIGHTS)

# Create an evaluator for the validation dataset
eval_output_dir = os.path.join(model_output_dir, "evaluation")
os.makedirs(eval_output_dir, exist_ok=True)
evaluator = COCOEvaluator("manga109_val", output_dir=eval_output_dir)

# Create a data loader for the validation set
val_loader = build_detection_test_loader(cfg, "manga109_val")

# Use the official `inference_on_dataset` tool
print("Running evaluation...")
results = inference_on_dataset(model, val_loader, evaluator)

# The results dictionary will also be printed
print("\nEvaluation results dictionary:")
print(results)

Running evaluation...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.762
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.926
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.901
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.617
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.786
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.077
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.633
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.804
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.703
 Average Recall  

In [8]:
import os
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
from detectron2.data.datasets import register_coco_instances
from detectron2.modeling import build_model
from detectron2.checkpoint import DetectionCheckpointer

# --- 1. Re-register the datasets (important if in a new script/session) ---
base_path = os.path.join(BASE_DIR, "data", "Manga109", "Human_Annotate_300")
images_dir = os.path.join(base_path, "train")
train_json_path = os.path.join(base_path, "train.json")
val_json_path = os.path.join(base_path, "val.json")

# Register the datasets
register_coco_instances("manga109_train", {}, train_json_path, images_dir)
register_coco_instances("manga109_val", {}, val_json_path, images_dir)


# --- 2. Setup Configuration ---
cfg = get_cfg()
# Use the same config file as for training

cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))

# Set the device
cfg.MODEL.DEVICE = str(device)

# Point to the model weights you just trained
model_output_dir = os.path.join(BASE_DIR, "models", "bubble-detection", "detectron2")
cfg.MODEL.WEIGHTS = os.path.join(model_output_dir, "model_final.pth")

# Set the number of classes (must match your training config)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # bubble

# Set the test dataset
cfg.DATASETS.TEST = ("manga109_val",)

# Set a confidence threshold for this evaluation
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.01  # You can adjust this threshold


# --- 3. Build the Model and Run Evaluation ---

# Build the model from the config
model = build_model(cfg)
# Load the weights
DetectionCheckpointer(model).load(cfg.MODEL.WEIGHTS)

# Create an evaluator for the validation dataset
eval_output_dir = os.path.join(model_output_dir, "evaluation")
os.makedirs(eval_output_dir, exist_ok=True)
evaluator = COCOEvaluator("manga109_val", output_dir=eval_output_dir)

# Create a data loader for the validation set
val_loader = build_detection_test_loader(cfg, "manga109_val")

# Use the official `inference_on_dataset` tool
print("Running evaluation...")
results = inference_on_dataset(model, val_loader, evaluator)

# The results dictionary will also be printed
print("\nEvaluation results dictionary:")
print(results)

Running evaluation...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.787
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.954
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.928
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.666
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.805
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.077
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.639
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.830
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.761
 Average Recall  